In [4]:
import pandas as pd 
import numpy as np 
import collections as cols

In [5]:
# keep this to 1 for outer metrices calculation else make it 0
outer = 0

In [6]:
qcount = 1008
if outer == 1:
    qstart = 0
    qlast = 1008
    start = 0
    last = qcount*3
    i = 1
else:
    qstart = 1008
    qlast = 1245
    start = qcount*3
    last = 3735
    qcount = 237
    i = 1009

In [7]:
df1 = pd.read_csv("Data/final.csv")[start:last]

In [8]:
# convert the to binary matrix 
def get_data(acolumn,right_ans):
    acolumn =  np.reshape(acolumn.values,(-1,3))
    repd = []
    for vec in acolumn: # vec = [2,3,1]
        rep = []
        for ind in vec:
            if ind in right_ans:
                rep.append(1)
            else:
                rep.append(0)
        repd.append(rep)
    return np.array(repd)

In [9]:
mat_bin = get_data(df1["Ans"],[1,2])
len(mat_bin)

237

In [10]:
def precision_at_k(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    prec = []
    for vec in data:
        rev_doc = 0
        tot_doc = 0
        prec_vec = []
        for doc in vec:
            if doc == 1:
                rev_doc +=1
            tot_doc +=1
            prec_vec.append(rev_doc/tot_doc)
        prec.append(np.array(prec_vec))
    return np.array(prec)

In [115]:
precision_at_k(mat_bin)[:10]

array([[0.        , 0.5       , 0.33333333],
       [1.        , 1.        , 0.66666667],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [11]:
def precision_at_3(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    prec_at_k = precision_at_k(mat_bin)
    k = 3-1
    prec_at_3 = [vec[k] for vec in prec_at_k]
    return np.array(prec_at_3)

In [117]:
precision_at_3(mat_bin)[:10]

array([0.33333333, 0.66666667, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [12]:
def avg_precision_at_3(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    data = precision_at_k(data)
    aprec = []
    for vec in data:
        aprec.append(sum(vec)/3)
    return np.array(aprec)

In [119]:
avg_precision_at_3(mat_bin)[:10]

array([0.27777778, 0.88888889, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [13]:
def reciprocal_rank(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    rr = []
    for vec in data: #  vec = [0,1,1]
        i = 1
        for ind in vec:
            if ind == 1:
                break
            i+=1
        #print(i)
        if i !=4:
            rr.append(1/i)
        else:
            rr.append(0)
    return np.array(rr)


In [121]:
reciprocal_rank(mat_bin)[:10]

array([0.5, 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

In [14]:
def success_ratio(data):
    """
    param : data
    2-D array representing right ans in each row
    """
    sr_mat = []
    for vec in data:
        sr_mat.append(sum(vec)/len(vec))
    
    return np.array(sr_mat)

In [123]:
success_ratio(mat_bin)[:10]

array([0.33333333, 0.66666667, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [15]:
def invert_rel(element):
    rmap = {1:2,2:1,3:0}
    return rmap[element] 

In [16]:
def ndcg(acolumn):
    """
    param : acolumn
    pandas series representing ans in each cell
    """
    idcg_vec = np.array([1/np.log2(2) , 1/np.log2(3) , 1/np.log2(4)])
    acolumn = acolumn.apply(invert_rel)
    acolumn =  np.reshape(acolumn.values,(qcount,3))

    ndcg_vec = []

    for vec in acolumn: #  vec = [1,2,1]   
        idcg = sum(idcg_vec*sorted(vec,reverse=True))
        if idcg!=0:
            ndcg_vec.append(np.sum(vec*idcg_vec)/idcg)
        else:
            ndcg_vec.append(0)
    return np.array(ndcg_vec)

In [126]:
ndcg(df1["Ans"])[:10]

array([0.63092975, 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [17]:
#get model number
def get_commom_entries(mod_index):
    data = pd.read_csv("final-Ques_rand.csv")[qstart:qlast]
    df = data[["Page","Model_name"]]
    models = mod_index
    print("MODELS : ",models)
    df["Model_num"] = df.Model_name.replace(models,[0,1,2])
    #df = df[entries]

    grp = {}
    dgrp = df.groupby(df.Model_num)
    grp["Grp1_Pg"] = dgrp.get_group(0).Page.values-i
    grp["Grp2_Pg"] = dgrp.get_group(1).Page.values-i
    grp["Grp3_Pg"] = dgrp.get_group(2).Page.values-i

    return grp

In [18]:
def grp_division(rows,mod_index):
    mod_num = get_commom_entries(mod_index)
    grp1 = rows[mod_num["Grp1_Pg"]]
    grp2 = rows[mod_num["Grp2_Pg"]]
    grp3 = rows[mod_num["Grp3_Pg"]]
    return grp1,grp2,grp3

In [19]:
def grp_mean(grp_mat):
    return (np.mean(grp_mat[0]),np.mean(grp_mat[1]),np.mean(grp_mat[2]))

In [20]:
def calc_metrics(df1,mod_index):
    """
    param : acolumn
    pandas series representing ans in each cell
    """
    acolumn = df1["Ans"] # we can also add df2 as in last step we will take common
    data = get_data(acolumn,[1,2])
    m_mat = {}

    m_mat["map"] = grp_mean(grp_division(avg_precision_at_3(data),mod_index))
    m_mat["mrr"] = grp_mean(grp_division(reciprocal_rank(data),mod_index))
    m_mat["avg_sr"] = grp_mean(grp_division(success_ratio(data),mod_index))
    m_mat["avg_ndcg"] = grp_mean(grp_division(ndcg(acolumn),mod_index))
    m_mat["avg_p3"] = grp_mean(grp_division(precision_at_3(data),mod_index))
    print(len(grp_division(precision_at_3(data),mod_index)[0]))
    return m_mat

In [21]:
mod_index=['गंगा', 'काली', 'रेनू']

In [22]:
calc_metrics(df1,mod_index)

MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']


/home/ritwik/envs/asha/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


MODELS :  ['गंगा', 'काली', 'रेनू']
79


{'map': (0.4135021097046413, 0.4240506329113924, 0.3396624472573839),
 'mrr': (0.5337552742616033, 0.5738396624472574, 0.4915611814345992),
 'avg_sr': (0.35443037974683544, 0.3544303797468354, 0.25316455696202533),
 'avg_ndcg': (0.5399821664174114, 0.581779639487739, 0.5076023181271029),
 'avg_p3': (0.35443037974683544, 0.3544303797468354, 0.25316455696202533)}

In [133]:
#For outer
if outer == 1:
    display(pd.DataFrame(calc_metrics(df1,mod_index),index=mod_index))

MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
336


,map,mrr,avg_sr,avg_ndcg,avg_p3
गंगा,0.183036,0.260913,0.163690,0.277396,0.163690
काली,0.219246,0.296627,0.199405,0.312296,0.199405
रेनू,0.105985,0.162202,0.087302,0.174693,0.087302


In [23]:
#For inner
if outer != 1:
    display(pd.DataFrame(calc_metrics(df1,mod_index),index=mod_index))

MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']
MODELS :  ['गंगा', 'काली', 'रेनू']


/home/ritwik/envs/asha/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


MODELS :  ['गंगा', 'काली', 'रेनू']
79


,map,mrr,avg_sr,avg_ndcg,avg_p3
गंगा,0.413502,0.533755,0.354430,0.539982,0.354430
काली,0.424051,0.573840,0.354430,0.581780,0.354430
रेनू,0.339662,0.491561,0.253165,0.507602,0.253165
